In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 910.5 kB/s eta 0:00:00


In [ ]:
import os
import requests
from pydantic import BaseModel, Field
from typing import Optional, List, Dict

# Ensure you have set your OpenAI API key
os.environ["OPENAI_API_KEY"] ="op..."

# --- 1. Pydantic Models for Data Validation ---

In [ ]:
class Employee(BaseModel):
    """A model to represent an employee's data for creation."""
    employee_id: str = Field(alias="Employee ID")
    first_name: str = Field(alias="First Name")
    last_name: str = Field(alias="Last Name")
    date_of_birth: str = Field(alias="Date of Birth")
    position: str = Field(alias="Position")
    department: str = Field(alias="Department")
    email: str = Field(alias="Email")
    phone_number: str = Field(alias="Phone Number")
    hire_date: str = Field(alias="Hire Date")
    salary: str = Field(alias="Salary")
    manager_name: Optional[str] = Field(None, alias="Manager Name")
    remaining_vacation_days: int = Field(alias="Remaining Vacation Days")

class EmployeeUpdate(BaseModel):
    """
    A model for partially updating an employee's data.
    All fields are optional, allowing for flexible updates.
    """
    first_name: Optional[str] = Field(None, description="The employee's first name.")
    last_name: Optional[str] = Field(None, description="The employee's last name.")
    position: Optional[str] = Field(None, description="The employee's job title.")
    salary: Optional[str] = Field(None, description="The employee's annual salary.")
    manager_name: Optional[str] = Field(None, description="The name of the employee's manager.")
    remaining_vacation_days: Optional[int] = Field(None, description="The number of remaining vacation days.")

In [ ]:
# --- 2. API Functions ---

BASE_URL = "https://kintern.app.n8n.cloud/webhook/api/employees"
MANIPULATION_BASE_URL = "https://kintern.app.n8n.cloud/webhook/885e0da1-8c2f-440f-ac0a-20b7a8bf977c/api/employee"

def create_employee(
    employee_id: str,
    first_name: str,
    last_name: str,
    date_of_birth: str,
    position: str,
    department: str,
    email: str,
    phone_number: str,
    hire_date: str,
    salary: str,
    remaining_vacation_days: int,
    manager_name: Optional[str] = None
) -> str:
    """
    Use this tool to create a new employee record. You must provide all required fields.
    """
    try:
        # Collect the function arguments into a dictionary to pass to the Pydantic model
        employee_dict = {k: v for k, v in locals().items() if v is not None}
        employee_data = Employee(**employee_dict)

        response = requests.post(BASE_URL, json=employee_data.model_dump(by_alias=True))
        response.raise_for_status()
        return f"Successfully created employee. Response: {response.json()}"
    except Exception as e:
        return f"An error occurred: {e}"


def get_all_employees() -> str:
    """
    Use this tool to get a list of all employees currently in the system.
    """
    try:
        response = requests.get(BASE_URL)
        response.raise_for_status()
        return f"Successfully retrieved all employees. Data: {response.json()}"
    except Exception as e:
        return f"An error occurred: {e}"

def get_one_employee(employee_id: str) -> str:
    """
    Use this tool to get the details of a single employee using their unique employee ID.only emplpoyee's ID is needed here
    employee_id:  The unique ID of the employee to retrieve.
    """
    try:
        url = f"{MANIPULATION_BASE_URL}/{employee_id}"
        response = requests.get(url)
        response.raise_for_status()
        return f"Successfully retrieved employee {employee_id}. Data: {response.json()}"
    except Exception as e:
        return f"An error occurred: {e}"

def update_one_employee(
    employee_id: str,
    first_name: Optional[str] = None,
    last_name: Optional[str] = None,
    position: Optional[str] = None,
    salary: Optional[str] = None,
    manager_name: Optional[str] = None,
    remaining_vacation_days: Optional[int] = None
) -> str:
    """
    Use this tool to update one or more details of a specific employee using their ID.
    Provide only the fields you wish to change.
    employee_id:  The unique ID of the employee to update.Mandatory for example : _6662225544
    """
    try:
        # Collect all provided optional arguments into a dictionary
        update_data = {k: v for k, v in locals().items() if k != 'employee_id' and v is not None}

        if not update_data:
            return "Error: You must provide at least one field to update."

        url = f"{MANIPULATION_BASE_URL}/{employee_id}"
        response = requests.patch(url, json=update_data)
        response.raise_for_status()
        return f"Successfully updated employee {employee_id}. Response: {response.json()}"
    except Exception as e:
        return f"An error occurred: {e}"

def delete_one_employee(employee_id: str) -> str:
    """
    Use this tool to delete a specific employee from the system using their unique employee ID.
    employee_id:  The unique ID of the employee to Delete.
    """
    try:
        url = f"{MANIPULATION_BASE_URL}/{employee_id}"
        response = requests.delete(url)
        response.raise_for_status()
        return f"Successfully deleted employee {employee_id}. Response: {response.json()}"
    except Exception as e:
        return f"An error occurred: {e}"

In [ ]:

# --- 3. Convert Functions to LangChain Tools ---
from langchain.tools import StructuredTool

# LangChain now infers the schema directly from the type-annotated function signatures.
# No manual 'args_schema' is needed.
tools = [
    StructuredTool.from_function(func=create_employee),
    StructuredTool.from_function(func=get_all_employees),
    StructuredTool.from_function(func=get_one_employee),
    StructuredTool.from_function(func=update_one_employee),
    StructuredTool.from_function(func=delete_one_employee),
]


In [ ]:
# --- 4. Initialize the LLM ---
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# --- 5. Create the ReAct Agent ---
from langchain import hub
from langchain.agents import create_react_agent

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)

# --- 6. Create an Agent Executor ---
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [ ]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [ ]:
print("--- Example 1: Getting all employees ---")
response = agent_executor.invoke({"input": "Can you show me a list of all employees in the company?"})
print("\nFinal Answer:", response["output"])

print("\n" + "="*50 + "\n")


--- Example 1: Getting all employees ---


> Entering new AgentExecutor chain...
To provide a list of all employees, I need to retrieve the current employee records from the system. 

Action: get_all_employees  
Action Input: None  Successfully retrieved all employees. Data: [{'id': 'recqM7bDV7OBEXYoe', 'createdTime': '2025-07-31T10:19:10.000Z', 'Employee ID': 'E001', 'First Name': 'Alice', 'Last Name': 'Smith', 'Date of Birth': '1988-02-14', 'Position': 'HR Manager', 'Department': 'Human Resources', 'Email': 'alice.smith@example.com', 'Phone Number': '+1-202-555-0141', 'Hire Date': '2015-06-01', 'Salary': 75000, 'Remaining Vacation Days': 12, 'Probation Completed': True}, {'id': 'recJ7NGC5pPnkdwZI', 'createdTime': '2025-07-31T10:19:10.000Z', 'Employee ID': 'E002', 'First Name': 'Bob', 'Last Name': 'Johnson', 'Date of Birth': '1990-07-23', 'Position': 'Software Engineer', 'Department': 'IT', 'Email': 'bob.johnson@example.com', 'Phone Number': '+1-202-555-0189', 'Hire Date': '2018-03-15

In [ ]:
print("--- Example 2: Getting a single employee ---")
response = agent_executor.invoke({
    "input": "I need to find the details for the employee with ID E008."
})
print("\nFinal Answer:", response["output"])

--- Example 2: Getting a single employee ---


> Entering new AgentExecutor chain...
I need to retrieve the details of the employee with the unique ID E008. 
Action: get_one_employee 
Action Input: "E008" Successfully retrieved employee E008. Data: {'id': 'rece9Zi5VUGrIS9Uz', 'createdTime': '2025-07-31T10:19:10.000Z', 'Employee ID': 'E008', 'First Name': 'Hannah', 'Last Name': 'White', 'Date of Birth': '1991-12-25', 'Position': 'Content Creator', 'Department': 'Marketing', 'Email': 'hannah.white@example.com', 'Phone Number': '+1-202-555-0155', 'Hire Date': '2022-02-01', 'Salary': 62000, 'Manager Name': 'Charlie Lee', 'Remaining Vacation Days': 14}I now have the details for the employee with ID E008. 

Final Answer: The employee with ID E008 is Hannah White, a Content Creator in the Marketing department. Her date of birth is December 25, 1991, and she was hired on February 1, 2022. Her salary is $62,000, and she has 14 remaining vacation days. Her manager is Charlie Lee, and her contact

In [ ]:
print("--- Example 3: Updating an employee ---")
response = agent_executor.invoke({
    "input": "Please update the position for employee E008 to 'Senior AI Developer' and their salary to '90000'."
})
print("\nFinal Answer:", response["output"])

print("\n" + "="*50 + "\n")

--- Example 3: Updating an employee ---


> Entering new AgentExecutor chain...
I need to update the position and salary for the employee with ID E008. I'll use the `update_one_employee` action to make these changes.

Action: update_one_employee  
Action Input: employee_id='E008', position='Senior AI Developer', salary='90000'  Error: You must provide at least one field to update.It seems that I need to provide at least one field to update, but I have already included both the position and salary. I will check if the employee ID E008 exists in the system to ensure that I am updating the correct employee.

Action: get_one_employee  
Action Input: employee_id='E008'  An error occurred: Expecting value: line 1 column 1 (char 0)It seems that there was an error when trying to retrieve the details of employee E008, which may indicate that this employee ID does not exist in the system. I will first retrieve the list of all employees to confirm whether E008 is present.

Action: get_all_employe

In [ ]:
# print("--- Example 4: Deleting an employee ---")
# response = agent_executor.invoke({
#     "input": "We need to remove employee E008 from the system."
# })
# print("\nFinal Answer:", response["output"])